In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("Brazil/BSLocations.csv")
df

,radio,mcc,mnc,lac,cid,changeable_0,lon,lat,range,sample,changeable=1,created,updated,avgsignal
0,GSM,724,11,5421,4242,0,-43.2387,-22.8463,10061,200,1,1344171639,1682933375,0
1,UMTS,724,11,44321,40728637,99,-43.2109,-22.8899,9193,51,1,1344173531,1682932904,0
2,UMTS,724,11,44321,40728639,115,-43.2123,-22.8973,9540,119,1,1344173531,1682932954,0
3,GSM,724,4,24141,13946,0,-49.2690,-25.5573,3041,175,1,1348409002,1690755408,0
4,GSM,724,4,24141,13944,0,-49.2460,-25.5626,9770,135,1,1348409002,1690755326,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34282,LTE,724,10,50512,231136023,-1,-45.5735,-23.4924,1000,4,1,1719170097,1719193327,0
34283,LTE,724,10,50512,231136021,-1,-45.5754,-23.4856,1000,23,1,1719170107,1719193327,0
34284,LTE,724,10,56012,231100683,-1,-45.7788,-23.2964,1000,6,1,1719171607,1719193330,0
34285,UMTS,724,6,40547,3113987,0,-48.6349,-26.9708,1000,1,1,1719160292,1719160292,0


In [5]:
df = pd.read_csv("/media/mohammed/PortableSSD/git/CoverageRateEstimation-main/Data/India/BSLocations.csv")
df = df.rename(columns={'long': 'lon'})
df.to_csv("Data/India/BSLocations.csv", index=False)
df

,radio,mcc,mnc,lac,cid,changeable_0,lon,lat,range,sample,changeable_1,created,updated,avgsignal
0,GSM,404,5,221,2171,0,70.380478,20.913162,1000,1,1,1459669222,1459669222,0
1,GSM,404,5,2312,12805,0,70.123672,21.126022,1000,1,1,1459669222,1459669222,0
2,GSM,404,69,441,40912,0,73.019943,19.039993,1000,1,1,1459760674,1459760674,0
3,GSM,404,45,25033,53132,0,77.543466,12.903442,1369,14,1,1459703141,1491232540,0
4,GSM,404,10,310,27942,0,77.236404,28.601303,1000,3,1,1459746500,1460953046,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810092,GSM,404,78,52012,17916,-1,77.793243,23.079309,1000,1,1,1681651479,1681651479,0
1810093,GSM,404,5,5704,45399,-1,72.597633,22.950689,1000,1,1,1681669306,1681669306,0
1810094,GSM,404,78,50244,1363,-1,77.793251,23.079281,1000,1,1,1681669519,1681669519,0
1810095,GSM,404,78,50244,1362,-1,77.793251,23.079281,1000,1,1,1681669519,1681669519,0


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [1]:
import torch
from torch import nn
from torch.nn import Unflatten
import matplotlib.pyplot as plt
class Encoder(nn.Module):
    def __init__(self, encoded_space_dim=8):
        super(Encoder, self).__init__()

        self.xLin = int(64/8)
        self.yLin = int(64/8)    # Determines input size of linear layer

        # Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 8, (3, 3), stride=(2, 2), padding=(1, 1)),
            nn.ReLU(True),
            nn.Conv2d(8, 16, (3, 3), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, (3, 3), stride=(2, 2), padding=(1, 1)),
            nn.ReLU(True)
        )

        # Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        # Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(self.xLin * self.yLin * 32, 128),
            nn.ReLU(True),
            nn.Linear(128, encoded_space_dim)
        )

    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        return x


class Decoder(nn.Module):

    def __init__(self, encoded_space_dim=8):
        super().__init__()

        self.xLin = int(64 / 8)
        self.yLin = int(64 / 8)  # Determines output size of linear layer

        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, self.xLin * self.yLin * 32),
            nn.ReLU(True)
        )

        self.unflatten = nn.Unflatten(dim=1,
                                      unflattened_size=(32, self.xLin, self.yLin))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(32, 16, (3, 3),
                               stride=(2, 2), padding=(1, 1), output_padding=(1, 1)),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, (3, 3), stride=(2, 2),
                               padding=(1, 1), output_padding=(1, 1)),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, (3, 3), stride=(2, 2),
                               padding=(1, 1), output_padding=(1, 1))
        )

    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

#Brazil

In [4]:
import torch
import numpy as np
# from NeuralNets import Encoder, Decoder
import matplotlib.pyplot as plt


def AM():                                 # Alternative Maximization
    # ModelsFolder = args.Models + args.Country + '/'
    ModelsFolder = 'C:/Users/asad/Downloads/min/min/Models/India/Shape1.0'
    SINRThrIndex = int((0 - (-5))/((20 - (-5))/25))
    BoxIndex = np.load(ModelsFolder + 'BoxIndex.npy')
    rel_x = torch.tensor(np.load('C:/Users/asad/Downloads/min/min/Models/India/Shape1.0/rel_x.npy'))
    rel_y = torch.tensor(np.load('C:/Users/asad/Downloads/min/min/Models/India/Shape1.0/rel_y.npy'))
    BoxCardinality = np.load(('C:/Users/asad/Downloads/min/min/Models/India/Shape1.0/BoxCardinality.npy')
    EndBSIndex = np.load('C:\Users\asad\Downloads\min\min\Models\India\Shape1.0\BoxIndex.npy')
    #EndBSIndex = np.load(('C:/Users/asad/Downloads/min/min/Models/India/Shape1.0/EndBSIndex.npy')
    xLength = 5 #args.xLength
    yLength = 5 #args.yLength
    xSteps = 32 #args.xSteps
    ySteps = 32 #args.ySteps

    BoxIndex = np.random.permutation(BoxIndex)
    ChosenBoxIndex = BoxIndex[0]                     # Randomly choose a box
    OldNumBS = BoxCardinality[ChosenBoxIndex]
    xBSOld = rel_x[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # x coordinates of existing BSs
    xCellsOld = (xBSOld / (xLength / xSteps)).long()
    yBSOld = rel_y[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # y coordinates of existing BSs
    yCellsOld = (yBSOld / (yLength / ySteps)).long()
    # xSteps = 64
    # ySteps = 64
    MaskRatio = 0.5
    UserFracThrInv = 0.9
    MaxDeployInv = 50
    MaxRandInitInv = 2
    batchlength = 10
    CoverageLBInv = 0.9
    # BSCells = torch.zeros(batchlength, 1, xSteps, ySteps)
    # print(BSCells.shape)
    EncNet = Encoder()
    EncNet.load_state_dict(torch.load(('C:/Users/asad/Downloads/min/min/Models/India/Shape1.0/Encoder24.pkl'))
    DecNet = Decoder()
    DecNet.load_state_dict(torch.load(('C:/Users/asad/Downloads/min/min/Models/India/Shape1.0/Decoder24.pkl'))

    def ComputeObj(BSCell):
        x = EncNet(BSCell)
        x = DecNet(x)
        x = x[0, 0, 0:32, 0:32].detach().numpy()

        return torch.mean(torch.tensor(x > CoverageLBInv)*1.0)

    def DrawHeatMap():
        x = EncNet(BSCells)
        x = DecNet(x)
        x = x[0, 0, 0:32, 0:32].detach().numpy()


        plt.figure(figsize=(7, 6))
        plt.pcolormesh(x)
        plt.colorbar()
        # plt.xlim([-int(xSteps/2) + int(int(xSteps / 2)/MaskRatio), int(xSteps/2) + int(int(xSteps / 2)/MaskRatio)])
        # plt.ylim([-int(ySteps/2) + int(int(ySteps / 2)/MaskRatio), int(ySteps/2) + int(int(ySteps / 2)/MaskRatio)])
        plt.xticks([])
        plt.yticks([])

        plt.scatter(xCellsOld - int(xSteps / 2) + int(int(xSteps / 2)/MaskRatio), yCellsOld - int(ySteps / 2) + int(int(ySteps / 2)/MaskRatio), marker='o', c='red')
        plt.scatter(xCellsNew - int(xSteps / 2) + int(int(xSteps / 2)/MaskRatio), yCellsNew - int(ySteps / 2) + int(int(ySteps / 2)/MaskRatio), marker='*', c='red')

        if NewNumBS == 0:
            plt.title('Before deployment')
            plt.savefig('India/' + f'/Old{OldNumBS}New{NewNumBS}.png')
            
        if Val >= UserFracThrInv or NewNumBS == MaxDeployInv - 1:
            plt.title('After deployment')
            plt.savefig('India/' + f'/Old{OldNumBS}New{NewNumBS}.png')
        plt.close()  # Close the figure after saving
        plt.close('all')
    # xLength = args.xLength
    # yLength = args.yLength
    # xSteps = args.xSteps
    # ySteps = args.ySteps

    BoxIndex = np.random.permutation(BoxIndex)
    ChosenBoxIndex = BoxIndex[0]                     # Randomly choose a box
    OldNumBS = BoxCardinality[ChosenBoxIndex]
    xBSOld = rel_x[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # x coordinates of existing BSs
    xCellsOld = (xBSOld / (xLength / xSteps)).long()
    yBSOld = rel_y[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # y coordinates of existing BSs
    yCellsOld = (yBSOld / (yLength / ySteps)).long()

    for NewNumBS in range(MaxDeployInv):
        print(NewNumBS)
        for RandInitIndex in range(MaxRandInitInv):
            print(RandInitIndex)
            BSCells = torch.zeros(1, 1, xSteps, ySteps)
            xCellsNew = torch.randint(0, xSteps, [NewNumBS])
            yCellsNew = torch.randint(0, ySteps, [NewNumBS])
            BSCells[0, 0, xCellsNew, yCellsNew] = 1            # Random Initialization for new BSs
            BSCells[0, 0, xCellsOld, yCellsOld] = 1

            Val = ComputeObj(BSCells)
            Improvement = True

            while Improvement:
                Improvement = False

                for BSIndex in range(NewNumBS):

                    # Maximization along X-axis
                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 0
                    for xIndex in range(xSteps):
                        BSCells[0, 0, xIndex, yCellsNew[BSIndex]] = 1
                        tempVal = ComputeObj(BSCells)
                        BSCells[0, 0, xIndex, yCellsNew[BSIndex]] = 0

                        if tempVal > Val:
                            Improvement = True
                            xCellsNew[BSIndex] = xIndex
                            Val = tempVal

                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 1

                    # Maximization along Y-axis
                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 0
                    for yIndex in range(ySteps):
                        BSCells[0, 0, xCellsNew[BSIndex], yIndex] = 1
                        tempVal = ComputeObj(BSCells)
                        BSCells[0, 0, xCellsNew[BSIndex], yIndex] = 0

                        if tempVal > Val:
                            Improvement = True
                            yCellsNew[BSIndex] = yIndex
                            Val = tempVal

                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 1

        # args.logger.info(f'Existing BSs: {OldNumBS},  New Deployed BSs: {NewNumBS}.')
        # args.logger.info(f'Maximum fraction of cells with coverage above the threshold is: {Val}.')

        DrawHeatMap()

        if Val >= UserFracThrInv:
            break


SyntaxError: invalid syntax (3683562280.py, line 15)

In [4]:
AM()

NameError: name 'AM' is not defined

In [59]:
import torch
import numpy as np
# from NeuralNets import Encoder, Decoder
import matplotlib.pyplot as plt


def AM():                                 # Alternative Maximization
    # ModelsFolder = args.Models + args.Country + '/'
    ModelsFolder = 'Brazil/Shape1.0/'
    SINRThrIndex = int((0 - (-5))/((20 - (-5))/25))
    BoxIndex = np.load('Brazil/Shape1.0/BoxCardinality.npy')
    rel_x = torch.tensor(np.load('Brazil/Shape1.0/rel_y.npy'))
    rel_y = torch.tensor(np.load('Models/Brazil/Shape1.0/rel_x.npy'))
    BoxCardinality = np.load('Models/Brazil/Shape1.0/BoxCardinality.npy')

    EndBSIndex = np.load('Models/Brazil/Shape1.0/EndBSIndex.npy')
    xLength = 10 #args.xLength
    yLength = 10 #args.yLength
    xSteps = 64 #args.xSteps
    ySteps = 64 #args.ySteps

    BoxIndex = np.random.permutation(BoxIndex)
    ChosenBoxIndex = BoxIndex[0]                     # Randomly choose a box
    OldNumBS = BoxCardinality[ChosenBoxIndex]
    xBSOld = rel_x[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # x coordinates of existing BSs
    xCellsOld = (xBSOld / (xLength / xSteps)).long()
    yBSOld = rel_y[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # y coordinates of existing BSs
    yCellsOld = (yBSOld / (yLength / ySteps)).long()
    # xSteps = 64
    # ySteps = 64
    MaskRatio = 0.5
    UserFracThrInv = 0.9
    MaxDeployInv = 4
    MaxRandInitInv = 3
    batchlength = 10
    CoverageLBInv = 0.9
    # BSCells = torch.zeros(batchlength, 1, xSteps, ySteps)
    # print(BSCells.shape)
    EncNet = Encoder()
    EncNet.load_state_dict(torch.load('Models/Brazil/Shape1.0/Encoder24.pkl'))
    DecNet = Decoder()
    DecNet.load_state_dict(torch.load('Models/Brazil/Shape1.0/Decoder24.pkl'))

    def ComputeObj(BSCell):
        x = EncNet(BSCell)
        x = DecNet(x)
        x = x[0, 0, 0:64, 0:64].detach().numpy()

        return torch.mean(torch.tensor(x > CoverageLBInv)*1.0)

    def DrawHeatMap():
        x = EncNet(BSCells)
        x = DecNet(x)
        x = x[0, 0, 0:64, 0:64].detach().numpy()


        plt.figure(figsize=(7, 6))
        plt.pcolormesh(x)
        plt.colorbar()
        # plt.xlim([-int(xSteps/2) + int(int(xSteps / 2)/MaskRatio), int(xSteps/2) + int(int(xSteps / 2)/MaskRatio)])
        # plt.ylim([-int(ySteps/2) + int(int(ySteps / 2)/MaskRatio), int(ySteps/2) + int(int(ySteps / 2)/MaskRatio)])
        plt.xticks([])
        plt.yticks([])

        plt.scatter(xCellsOld - int(xSteps / 2) + int(int(xSteps / 2)/MaskRatio), yCellsOld - int(ySteps / 2) + int(int(ySteps / 2)/MaskRatio), marker='o', c='red')
        plt.scatter(xCellsNew - int(xSteps / 2) + int(int(xSteps / 2)/MaskRatio), yCellsNew - int(ySteps / 2) + int(int(ySteps / 2)/MaskRatio), marker='*', c='red')

        if NewNumBS == 0:
            plt.title('Before deployment')
            plt.savefig('/Results/Brazil' + f'/Old{OldNumBS}New{NewNumBS}.png')
            # plt.close()  # Close the figure after saving
            # plt.close('all') 
        if Val >= UserFracThrInv or NewNumBS == MaxDeployInv - 1:
            plt.title('After deployment')
            plt.savefig('Results/Brazil' + f'/Old{OldNumBS}New{NewNumBS}.png')
            # plt.close()  # Close the figure after saving
            # plt.close('all')
        plt.close()  # Close the figure after saving
        plt.close('all') 
    # xLength = args.xLength
    # yLength = args.yLength
    # xSteps = args.xSteps
    # ySteps = args.ySteps

    BoxIndex = np.random.permutation(BoxIndex)
    ChosenBoxIndex = BoxIndex[0]                     # Randomly choose a box
    OldNumBS = BoxCardinality[ChosenBoxIndex]
    xBSOld = rel_x[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # x coordinates of existing BSs
    xCellsOld = (xBSOld / (xLength / xSteps)).long()
    yBSOld = rel_y[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # y coordinates of existing BSs
    yCellsOld = (yBSOld / (yLength / ySteps)).long()

    for NewNumBS in range(MaxDeployInv):
        print(NewNumBS)
        for RandInitIndex in range(MaxRandInitInv):
            print(RandInitIndex)
            BSCells = torch.zeros(1, 1, xSteps, ySteps)
            xCellsNew = torch.randint(0, xSteps, [NewNumBS])
            yCellsNew = torch.randint(0, ySteps, [NewNumBS])
            BSCells[0, 0, xCellsNew, yCellsNew] = 1            # Random Initialization for new BSs
            BSCells[0, 0, xCellsOld, yCellsOld] = 1

            Val = ComputeObj(BSCells)
            Improvement = True

            while Improvement:
                Improvement = False

                for BSIndex in range(NewNumBS):

                    # Maximization along X-axis
                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 0
                    for xIndex in range(xSteps):
                        BSCells[0, 0, xIndex, yCellsNew[BSIndex]] = 1
                        tempVal = ComputeObj(BSCells)
                        BSCells[0, 0, xIndex, yCellsNew[BSIndex]] = 0

                        if tempVal > Val:
                            Improvement = True
                            xCellsNew[BSIndex] = xIndex
                            Val = tempVal

                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 1

                    # Maximization along Y-axis
                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 0
                    for yIndex in range(ySteps):
                        BSCells[0, 0, xCellsNew[BSIndex], yIndex] = 1
                        tempVal = ComputeObj(BSCells)
                        BSCells[0, 0, xCellsNew[BSIndex], yIndex] = 0

                        if tempVal > Val:
                            Improvement = True
                            yCellsNew[BSIndex] = yIndex
                            Val = tempVal

                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 1

        # args.logger.info(f'Existing BSs: {OldNumBS},  New Deployed BSs: {NewNumBS}.')
        # args.logger.info(f'Maximum fraction of cells with coverage above the threshold is: {Val}.')

        DrawHeatMap()

        if Val >= UserFracThrInv:
            break


In [60]:
AM()

0
0
1
2
1
0
1
2
2
0
1
2
3
0
1
2


In [ ]:
import torch
import numpy as np
from NeuralNets import Encoder, Decoder
import matplotlib.pyplot as plt


def AM(args):                                 # Alternative Maximization
    # ModelsFolder = args.Models + args.Country + '/'
    ModelsFolder = args.Models + args.Country + f'/Shape{args.m_par}/'
    SINRThrIndex = int((args.SINRThrInv - args.MinSINRThr)/((args.MaxSINRThr - args.MinSINRThr)/args.NumSINRThr))

    if (SINRThrIndex < 0) or (SINRThrIndex >= args.NumSINRThr):
        raise ValueError('Chosen SINR threshold for the inverse problem is outside of training range.')

    # Load Data
    rel_x = torch.tensor(np.load(ModelsFolder + 'rel_x.npy'))
    rel_y = torch.tensor(np.load(ModelsFolder + 'rel_y.npy'))
    BoxIndex = np.load(ModelsFolder + 'BoxIndex.npy')
    BoxCardinality = np.load(ModelsFolder + 'BoxCardinality.npy')
    EndBSIndex = np.load(ModelsFolder + 'EndBSIndex.npy')

    EncNet = Encoder(args)
    EncNet.load_state_dict(torch.load(ModelsFolder + f'Encoder{SINRThrIndex}.pkl'))
    DecNet = Decoder(args)
    DecNet.load_state_dict(torch.load(ModelsFolder + f'Decoder{SINRThrIndex}.pkl'))

    def ComputeObj(BSCell):
        x = EncNet(BSCell)
        x = DecNet(x)
        x = x[0, 0, args.xIndexMin:args.xIndexMax, args.yIndexMin:args.yIndexMax]
        return torch.mean(torch.tensor(x > args.CoverageLBInv)*1.0)

    def DrawHeatMap():
        x = EncNet(BSCells)
        x = DecNet(x)
        x = x[0, 0, args.xIndexMin:args.xIndexMax, args.yIndexMin:args.yIndexMax].detach().numpy()

        plt.figure(figsize=(7, 6))
        plt.pcolormesh(x)
        plt.colorbar()
        plt.xlim([-int(args.xSteps/2) + int(int(args.xSteps / 2)/args.MaskRatio), int(args.xSteps/2) + int(int(args.xSteps / 2)/args.MaskRatio)])
        plt.ylim([-int(args.ySteps/2) + int(int(args.ySteps / 2)/args.MaskRatio), int(args.ySteps/2) + int(int(args.ySteps / 2)/args.MaskRatio)])
        plt.xticks([])
        plt.yticks([])

        plt.scatter(xCellsOld - int(args.xSteps / 2) + int(int(args.xSteps / 2)/args.MaskRatio), yCellsOld - int(args.ySteps / 2) + int(int(args.ySteps / 2)/args.MaskRatio), marker='o', c='red')
        plt.scatter(xCellsNew - int(args.xSteps / 2) + int(int(args.xSteps / 2)/args.MaskRatio), yCellsNew - int(args.ySteps / 2) + int(int(args.ySteps / 2)/args.MaskRatio), marker='*', c='red')

        if NewNumBS == 0:
            plt.title('Before deployment')
            plt.savefig('Results/Inverse/' + args.Country + f'/Old{OldNumBS}New{NewNumBS}.png')
            plt.close()  # Close the figure after saving

        if Val >= args.UserFracThrInv or NewNumBS == args.MaxDeployInv - 1:
            plt.title('After deployment')
            plt.savefig('Results/Inverse/' + args.Country + f'/Old{OldNumBS}New{NewNumBS}.png')
            plt.close()  # Close the figure after saving

    xLength = args.xLength
    yLength = args.yLength
    xSteps = args.xSteps
    ySteps = args.ySteps

    BoxIndex = np.random.permutation(BoxIndex)
    ChosenBoxIndex = BoxIndex[0]                     # Randomly choose a box
    OldNumBS = BoxCardinality[ChosenBoxIndex]
    xBSOld = rel_x[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # x coordinates of existing BSs
    xCellsOld = (xBSOld / (xLength / xSteps)).long()
    yBSOld = rel_y[EndBSIndex[ChosenBoxIndex] - OldNumBS: EndBSIndex[ChosenBoxIndex]]  # y coordinates of existing BSs
    yCellsOld = (yBSOld / (yLength / ySteps)).long()

    for NewNumBS in range(args.MaxDeployInv):
        for RandInitIndex in range(args.MaxRandInitInv):
            BSCells = torch.zeros(1, 1, xSteps, ySteps)
            xCellsNew = torch.randint(0, xSteps, [NewNumBS])
            yCellsNew = torch.randint(0, ySteps, [NewNumBS])
            BSCells[0, 0, xCellsNew, yCellsNew] = 1            # Random Initialization for new BSs
            BSCells[0, 0, xCellsOld, yCellsOld] = 1

            Val = ComputeObj(BSCells)
            Improvement = True

            while Improvement:
                Improvement = False

                for BSIndex in range(NewNumBS):

                    # Maximization along X-axis
                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 0
                    for xIndex in range(xSteps):
                        BSCells[0, 0, xIndex, yCellsNew[BSIndex]] = 1
                        tempVal = ComputeObj(BSCells)
                        BSCells[0, 0, xIndex, yCellsNew[BSIndex]] = 0

                        if tempVal > Val:
                            Improvement = True
                            xCellsNew[BSIndex] = xIndex
                            Val = tempVal

                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 1

                    # Maximization along Y-axis
                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 0
                    for yIndex in range(ySteps):
                        BSCells[0, 0, xCellsNew[BSIndex], yIndex] = 1
                        tempVal = ComputeObj(BSCells)
                        BSCells[0, 0, xCellsNew[BSIndex], yIndex] = 0

                        if tempVal > Val:
                            Improvement = True
                            yCellsNew[BSIndex] = yIndex
                            Val = tempVal

                    BSCells[0, 0, xCellsNew[BSIndex], yCellsNew[BSIndex]] = 1

        args.logger.info(f'Existing BSs: {OldNumBS},  New Deployed BSs: {NewNumBS}.')
        args.logger.info(f'Maximum fraction of cells with coverage above the threshold is: {Val}.')

        DrawHeatMap()

        if Val >= args.UserFracThrInv:
            break
